# Importations

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from torchvision import models
import torch.nn as nn
from timm import create_model

## Loading the dataset

In [2]:
dataset_path = "./img_dataset"

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(root=dataset_path, transform=preprocess)

### Splitting the dataset into a train and a test dataset

In [3]:
train_dataset, test_dataset = random_split(
    dataset=dataset,
    lengths=[int(0.8*len(dataset)), int(0.2*len(dataset))+1]
)

In [4]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

## Usage of a foundation model

In [10]:
# Adapter la dernière couche au nombre de classes
num_classes = len(dataset.classes)
vit_model = "vit_base_patch16_224"
swin_model = "swin_base_patch4_window7_224"
model = create_model(swin_model, pretrained=True, num_classes=num_classes)
# model.fc = nn.Linear(model.fc.in_features, num_classes)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
for param in model.parameters():
    param.requires_grad = True

# Training and evaluating the model

In [14]:
# Définir les paramètres
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Boucle d'entraînement
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()  # Mode entraînement
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

        # Évaluer après chaque epoch
        evaluate_model(model, test_loader)

# Fonction d'évaluation
def evaluate_model(model, test_loader):
    model.eval()  # Mode évaluation
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Lancer l'entraînement
train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5)


Epoch [1/5], Loss: 2.1595
Test Accuracy: 27.52%
Epoch [2/5], Loss: 1.9391
Test Accuracy: 27.52%
Epoch [3/5], Loss: 1.9107
Test Accuracy: 30.28%
Epoch [4/5], Loss: 1.8851
Test Accuracy: 30.28%
Epoch [5/5], Loss: 1.8740
Test Accuracy: 30.28%
